# Modelado usando ARIMA

**Objetivo:** Seleccionar y aplicar un modelo predictivo para estimar la frecuencia de los accidentes.

## ARIMA
ARIMA es un modelo económico para predecir series de tiempo, es decir, mediciones capturadas en un punto específico del tiempo y para los cuales exitste algún tipo de autocorrelación donde el valor actual x explica de alguna forma al valor x+1.

El nombre ARIMA significa modelo autorregresivo integrado de media móvil (del inglés AutoRegressive Integrated Moving Average) y está compuesto por varios parámetros.

### AR
La autoregresión (AR) modela la siguiente observación (x+1) de una serie de tiempo en términos de una relación lineal de *n* observaciones anteriores. 

Los modelos AR reciben por parámetro un valor *p* que determina el orden de la relación lineal.

### MA
Las medias móviles (MA) modela la siguiente observación (x+1) de una serie de tiempo en términos de una relación lineal de la media de los errores calculados en un determinado número de observaciones *q*.

El modelo de media móvil recibe dicho valor *q* como parametro.

### I
Los modelos ARIMA conbinan ambos modelos, los autoregresivos (AR) y los de media móvil (MA) añadiendo un componente llamado integración (I) que cumple la función de transformar la serie en función de su estacionalidad. 

Este componente recibe por parámetro un número *d* de observaciones previas a los que transforma.

## SARIMA
Dada la complejidad de la serie de tiempo de los accidentes de la ciudad de Barranquilla, se utiliza tambien una variante del modelo ARIMA que incluye un componente adicional para modelar la estacionalidad en la que se tiene en cuenta el error de valores que se repiten con alguna frecuencia dada para mejorar la precisión de la predicción.

## Implementación de modelos en Python
Para implementar los modelos ARIMA en Python es necesario primero tener los datos correctos, para ello se construye el dataset que expresa la frecuencia diaria de los accidentes de tráfico en Barranquilla desde 2015 hasta 2018.

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('../../data/processed/Accidentes_clean.csv')
df['FECHA_ACCIDENTE'] = pd.to_datetime(df['FECHA_ACCIDENTE'], format="%Y-%m-%d %H:%M:%S")
daily_data = df.set_index('FECHA_ACCIDENTE').groupby(pd.Grouper(freq='d')).agg({'CANTIDAD_ACCIDENTES':sum}).reset_index()
daily_data.head()

,FECHA_ACCIDENTE,CANTIDAD_ACCIDENTES
0,2015-01-01,10
1,2015-01-02,13
2,2015-01-03,11
3,2015-01-04,7
4,2015-01-05,14


Luego, se divide el dataset de tal forma que se tenga sólo los datos de entrenamiento, para este caso se escogen todos los registros desde 2015 hasta mayo de 2018.

In [2]:
train = daily_data.set_index('FECHA_ACCIDENTE').loc[:'2017-12-31']
train.tail()

,CANTIDAD_ACCIDENTES
FECHA_ACCIDENTE,
2017-12-27,16
2017-12-28,21
2017-12-29,9
2017-12-30,10
2017-12-31,8


Una vez se tiene el dataset de entrenamiento se procede a crear los modelos utilizando pmdarima, que es el equivalente en Python al popular módulo *AutoARIMA* del lenguaje estadístico R.

In [3]:
from pmdarima.arima import auto_arima

stepwise_fit = auto_arima(train, start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                          start_P=0, seasonal=True, d=1, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

stepwise_fit.summary()

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=6878.203, BIC=6903.141, Fit time=7.730 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=8372.726, BIC=8382.701, Fit time=0.140 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=7726.022, BIC=7745.972, Fit time=0.912 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=6879.902, BIC=6899.852, Fit time=6.140 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=6862.987, BIC=6892.912, Fit time=7.093 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 0, 12); AIC=7287.713, BIC=7312.651, Fit time=5.508 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=6878.126, BIC=6913.038, Fit time=24.778 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=7691.538, BIC=7711.488, Fit time=1.807 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(2, 1, 2, 12); AIC=6847.179, BIC=6887.078, Fit time=32.441 seconds
Fit ARIMA: order=(0, 1, 1)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                 1096
Model:             SARIMAX(0, 1, 1)x(2, 1, 2, 12)   Log Likelihood               -3415.690
Date:                            Mon, 15 Apr 2019   AIC                           6845.380
Time:                                    18:55:58   BIC                           6880.292
Sample:                                         0   HQIC                          6858.597
                                           - 1096                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0005      0.001     -0.812      0.417      -0.002       0.001
ma.L1         -0.9555      0.010    -95.031      0.000      -0.975      -0.936
ar.S.L12      -1.0547      0.051    -20.715      0.000      -1.154      -0.955
ar.S.L24      -0.1918      0.031     -6.169      0.000      -0.253      -0.131
ma.S.L12      -0.0801      0.052     -1.538      0.124      -0.182       0.022
ma.S.L24      -0.9060      0.056    -16.272      0.000      -1.015      -0.797
sigma2        30.4857      1.669     18.266      0.000      27.215      33.757
===================================================================================
Ljung-Box (Q):                      313.97   Jarque-Bera (JB):                 6.20
Prob(Q):                              0.00   Prob(JB):                         0.04
Heteroskedasticity (H):               1.23   Skew:                             0.07
Prob(H) (two-sided):                  0.05   Kurtosis:                         2.66
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Finalmente, se guarda el modelo entrenado en la carpeta model.

In [6]:
import pickle
filename = '../../model/test models/SARIMA_model.sav'
pickle.dump(stepwise_fit, open(filename, 'wb'))